# CodeCraft Code Quality Fixes Guide

## Complete Reference for Fixing Linting, Type-Checking, and Code Quality Issues

This notebook provides a comprehensive guide to understanding and fixing code quality issues reported by Pylint, Flake8, and Mypy in the CodeCraft repository.

### Tools Overview:
- **Flake8**: PEP 8 style compliance and basic errors
- **Pylint**: Comprehensive code analysis (imports, conventions, standards)
- **Mypy**: Static type checking and type annotation validation

## Section 1: Understanding Code Quality Warnings and Errors

### Severity Levels

| Level | Meaning | Examples |
|-------|---------|----------|
| **Error** (8) | Breaking issues that must be fixed | E402 (module level import not at top), F841 (unused variable), E1101 (no member) |
| **Warning** (4) | Important issues to address | W0201 (attribute-defined-outside-init), W0613 (unused-argument), W1510 (subprocess-run-check) |
| **Convention** (2) | Style and readability improvements | C0301 (line-too-long), C0302 (too-many-lines), C0303 (trailing-whitespace) |
| **Info/Note** (2) | Hints and suggestions | annotation-unchecked, import-untyped |

### Common Issue Categories in CodeCraft

1. **Import Organization** - Imports not at module top level
2. **Attribute Definition** - Instance attributes defined outside `__init__`
3. **Type Checking** - Union types, missing type hints, untyped functions
4. **Code Style** - Line length, trailing whitespace, blank lines
5. **Unused Code** - Unused imports, unused variables, dead code
6. **Exception Handling** - Overly broad exception catching

## Section 2: Fixing Import Organization Issues (E402, C0413)

### Problem: Module Level Imports Not at Top of File

**Issue**: When module-level code or other imports appear before certain imports, Flake8 and Pylint flag E402/C0413 errors.

### Example Issues Found in CodeCraft

**File**: `src/codex/codex.py` (line 26)
```python
# BEFORE (WRONG)
import tkinter as tk
import sys
from pathlib import Path

sys.path.insert(0, str(Path(__file__).parent.parent))

from codex.codex_gui import CodeExIDE  # E402: module level import not at top
```

**File**: `src/codex/codex_gui.py` (lines 34-44)
```python
# BEFORE (WRONG)
import tkinter as tk
# ... other imports ...
import sys
from pathlib import Path

sys.path.insert(0, str(Path(__file__).parent.parent))  # Module-level code!

from hb_lcs.language_config import LanguageConfig  # E402: module level import not at top
```

### Solution: Reorganize Imports Before Other Code

```python
# AFTER (CORRECT)
import tkinter as tk
import sys
from pathlib import Path
from typing import Optional, Dict, Any
from datetime import datetime

# Ensure parent modules are in path
sys.path.insert(0, str(Path(__file__).parent.parent))

# Now import modules that depend on path modification
from hb_lcs.language_config import LanguageConfig
from hb_lcs.interpreter_generator import (
    InterpreterGenerator,
    InterpreterPackage,
)
```

### Key Rules for Import Organization:
1. **All imports must come before any module-level code**
2. **Order of imports**: stdlib → third-party → local modules
3. **Group related imports together**
4. **Use parentheses for multi-line imports**

## Section 3: Resolving Attribute Definition Problems (W0201)

### Problem: Attributes Defined Outside __init__

**Issue**: When instance attributes are created in methods other than `__init__`, Pylint warns W0201. This makes the class harder to understand as the full interface isn't clear from the constructor.

### CodeCraft Examples

**File**: `src/codex/codex_gui.py` (lines 116-148)
```python
# BEFORE (WRONG)
class CodeExIDE(ttk.Frame):
    def __init__(self, master):
        super().__init__(master)
        # Some initialization...
    
    def _setup_ui(self):
        self.interpreter_var = tk.StringVar(value="Select...")  # W0201!
        self.interpreter_combo = ttk.Combobox(...)  # W0201!
        self.status_label = ttk.Label(...)  # W0201!
        self.project_label = ttk.Label(...)  # W0201!
```

### Solution: Initialize All Attributes in __init__

```python
# AFTER (CORRECT)
class CodeExIDE(ttk.Frame):
    def __init__(self, master):
        super().__init__(master)
        # Initialize all attributes here
        self.interpreter_var: Optional[tk.StringVar] = None
        self.interpreter_combo: Optional[ttk.Combobox] = None
        self.status_label: Optional[ttk.Label] = None
        self.project_label: Optional[ttk.Label] = None
        
        # Then setup UI
        self._setup_ui()
    
    def _setup_ui(self):
        self.interpreter_var = tk.StringVar(value="Select...")
        self.interpreter_combo = ttk.Combobox(...)
        self.status_label = ttk.Label(...)
        self.project_label = ttk.Label(...)
```

### Best Practices:
- **All instance attributes should be listed in `__init__`**
- **Use type hints to document attribute types**
- **Assign `None` as initial value if created later**
- **Makes code more maintainable and type-checker friendly**

## Section 4: Handling Line Length Violations (C0301)

### Problem: Lines Exceeding 100 Characters

**Issue**: Lines longer than 100 characters reduce readability. Pylint warns C0301 when lines are too long.

### CodeCraft Examples

**File**: `src/codex/codex_gui.py` (line 109 - 101 chars)
```python
# BEFORE (TOO LONG)
self.root.geometry(self.settings.get("geometry", "1400x900"))  # 101 chars
```

**File**: `src/hb_lcs/ide.py` (line 1346 - 111 chars)
```python
# BEFORE (TOO LONG)
code = self.editor.get("1.0", tk.END).strip()  # Plus context makes it long
```

### Solution: Break Long Lines Strategically

```python
# AFTER (CORRECT) - Assign to variable first
default_geometry = self.settings.get("geometry", "1400x900")
self.root.geometry(default_geometry)

# AFTER (CORRECT) - Method chaining
code = self.editor.get("1.0", tk.END)
code = code.strip()

# AFTER (CORRECT) - Parenthesized continuation
result = (
    some_long_function_name(
        param1, param2, param3
    )
)

# AFTER (CORRECT) - Dictionary/List on multiple lines
config = {
    "name": "MyLanguage",
    "version": "1.0.0",
    "keywords": ["if", "then", "else"],
}
```

### Techniques:
1. **Extract to intermediate variables** - Split into steps
2. **Use parentheses for implicit line continuation**
3. **Break after commas in function arguments**
4. **Move string concatenation to multiple lines**
5. **Use backslash only as last resort**

## Section 5: Type Checking and Mypy Errors

### Problem: Union Types and Missing Type Guards

**Issue**: Mypy reports `union-attr` errors when accessing attributes on optional/union types without null checking.

### CodeCraft Examples

**Type**: `union-attr` Error
```python
# BEFORE (WRONG)
self.editor.delete("1.0", "end")  # Mypy: Item "None" of "Text | None" has no attribute "delete"
# self.editor is typed as Text | None, but we're accessing without checking

# BEFORE (WRONG)
self.notebook.add(frame, text="Tab")  # Mypy: Item "None" of "Notebook | None" has no attribute "add"
```

### Solution 1: Add Null Check (Type Guard)

```python
# AFTER (CORRECT)
if self.editor is not None:
    self.editor.delete("1.0", "end")

if self.notebook is not None:
    self.notebook.add(frame, text="Tab")
```

### Solution 2: Use Type Assertion (When You're Certain)

```python
# AFTER (CORRECT) - When you know it's not None
self.editor.delete("1.0", "end")  # type: ignore # or cast

# Better: Use cast from typing
from typing import cast
editor = cast(tk.Text, self.editor)
editor.delete("1.0", "end")
```

### Solution 3: Proper Type Hints in __init__

```python
# AFTER (CORRECT) - Initialize properly typed
from typing import Optional

class MyClass:
    def __init__(self):
        self.editor: tk.Text  # Will be assigned
        self.notebook: ttk.Notebook  # Will be assigned
        self._create_ui()
    
    def _create_ui(self):
        self.editor = tk.Text(...)
        self.notebook = ttk.Notebook(...)
```

### Common Mypy Issues in CodeCraft:

| Error | Cause | Solution |
|-------|-------|----------|
| `union-attr` | Accessing attribute on `Type \| None` | Add null check or type assertion |
| `attr-defined` | Attribute not defined in `__init__` | Initialize in constructor |
| `assignment` | Type mismatch in assignment | Add proper type hints |
| `annotation-unchecked` | Untyped function bodies | Add type hints or use `--check-untyped-defs` |

## Section 6: Cleaning Up Unused Code

### Problem: Unused Imports, Variables, and Dead Code

**Issue**: F401/W0611 (unused import), F841/W0612 (unused variable), F541 (f-string missing placeholders)

### CodeCraft Examples

**File**: `tests/test_teachscript.py` (line 9)
```python
# BEFORE (WRONG)
import os  # F401: imported but unused

import subprocess
result = subprocess.run(["python", "test.py"])  # W1510: no 'check' parameter
```

**File**: `tests/integration/test_ide.py` (line 17)
```python
# BEFORE (WRONG)
ide = CodeExIDE(root)  # F841: assigned but never used
```

**File**: `tests/test_teachscript.py` (line 78)
```python
# BEFORE (WRONG)
print(f"Test:")  # F541: f-string missing placeholders - should be plain string
```

### Solutions

```python
# AFTER (CORRECT) - Remove unused imports
import subprocess

# AFTER (CORRECT) - Use the variable
ide = CodeExIDE(root)
ide.run()  # Actually use it

# AFTER (CORRECT) - Fix subprocess.run
result = subprocess.run(["python", "test.py"], check=True)

# AFTER (CORRECT) - Convert to regular string if no placeholders
print("Test:")

# AFTER (CORRECT) - Use f-string only with variables
value = 42
print(f"Test: {value}")
```

### Quick Fixes:

1. **Unused Import**: Simply delete the import line
2. **Unused Variable**: Either use it or delete it
3. **F-string without placeholders**: Use regular string `"text"` instead of `f"text"`
4. **Missing subprocess 'check'**: Add `check=True` for error handling

### Tool to Help: Run Flake8
```bash
flake8 --select=F,E src/  # Shows unused imports and basic errors
```

## Section 7: Best Practices for Code Quality

### Comprehensive Checklist for Production-Ready Code

#### 1. **Type Hints and Annotations**
```python
# GOOD
def process_config(config: Dict[str, Any], 
                  debug: bool = False) -> Optional[str]:
    """Process language configuration.
    
    Args:
        config: Configuration dictionary
        debug: Enable debug output
        
    Returns:
        Processed configuration string or None if invalid
    """
    if not config:
        return None
    return str(config)

# Also annotate class attributes
class LanguageConfig:
    name: str
    version: str
    keywords: List[str]
    
    def __init__(self, name: str, version: str):
        self.name = name
        self.version = version
        self.keywords = []
```

#### 2. **Exception Handling**
```python
# BAD
try:
    result = dangerous_operation()
except Exception as e:  # Too broad!
    print(f"Error: {e}")

# GOOD
try:
    result = dangerous_operation()
except (ValueError, TypeError) as e:  # Specific exceptions
    logger.error(f"Invalid input: {e}")
except FileNotFoundError as e:
    logger.error(f"File missing: {e}")
except Exception as e:  # Only if absolutely needed
    logger.error(f"Unexpected error: {e}", exc_info=True)
```

#### 3. **Import Organization**
```python
# Standard library imports
import json
import sys
from pathlib import Path
from typing import Dict, List, Optional

# Third-party imports
import numpy as np
from flask import Flask

# Local imports
from .config import LanguageConfig
from .runtime import LanguageRuntime
```

#### 4. **Docstrings and Comments**
```python
def validate_keyword(name: str, reserved_words: Set[str]) -> bool:
    """Check if a name is a valid keyword.
    
    Args:
        name: The identifier to validate
        reserved_words: Set of reserved keyword names
        
    Returns:
        True if valid (not reserved), False otherwise
        
    Raises:
        ValueError: If name is empty
        
    Examples:
        >>> validate_keyword("if", {"if", "then"})
        False
        >>> validate_keyword("myvar", {"if", "then"})
        True
    """
    if not name:
        raise ValueError("Name cannot be empty")
    
    # Check against reserved words
    return name not in reserved_words
```

#### 5. **Naming Conventions**
```python
# GOOD - Clear, descriptive names
class KeywordMapping:
    def __init__(self, original: str, custom: str):
        self.original = original
        self.custom = custom
    
    def is_valid(self) -> bool:
        return bool(self.original and self.custom)

# BAD - Vague names
class KW:
    def __init__(self, o, c):
        self.o = o
        self.c = c
    
    def valid(self):
        return self.o and self.c
```

### Linting Configuration

**Create `.pylintrc` or update `pyproject.toml`:**
```ini
[tool.pylint.messages_control]
disable=
    missing-docstring,
    too-many-arguments,
    too-few-public-methods

[tool.pylint.format]
max-line-length=100

[tool.pylint.design]
max-attributes=10
max-args=8
```

### Pre-commit Hook Setup

```bash
# Install pre-commit
pip install pre-commit

# Create .pre-commit-config.yaml
cat > .pre-commit-config.yaml << 'EOF'
repos:
  - repo: https://github.com/psf/black
    rev: 23.1.0
    hooks:
      - id: black
        language_version: python3.10
  
  - repo: https://github.com/PyCQA/flake8
    rev: 6.0.0
    hooks:
      - id: flake8
        args: ['--max-line-length=100']
  
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.0.1
    hooks:
      - id: mypy
        additional_dependencies: ['types-all']
EOF

# Install the hook
pre-commit install
```

### Continuous Monitoring Workflow

1. **Before committing**: Run linters locally
   ```bash
   flake8 src/
   pylint src/
   mypy src/
   ```

2. **In CI/CD**: Add linting stage
   ```yaml
   - name: Lint code
     run: |
       flake8 src/ --max-line-length=100
       pylint src/ --rcfile=.pylintrc
       mypy src/ --strict
   ```

3. **Regular reviews**: Check linter reports and fix issues systematically